Download the precompiled `llama-server` binary from the following github repository

In [ ]:
!git clone https://github.com/jean-rl/llama-cpp-colab.git
%cd llama-cpp-colab
!chmod +x llama-server

Then, download model weights in GGUF format

In [ ]:
!wget https://huggingface.co/bartowski/Meta-Llama-3.1-8B-Instruct-GGUF/resolve/main/Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf

Then open a terminal and run the following command.

```
./llama-server -m Meta-Llama-3.1-8B-Instruct-Q4_K_M.gguf --port 9090 -v --verbose-prompt --jinja -ngl 9999
```

In [ ]:
import openai

In [ ]:
client = openai.OpenAI(
    base_url="http://127.0.0.1:9090/v1",
    api_key = "sk-no-key-required"
)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "user", "content": "Hi, how are you?"},
    {"role": "assistant", "content": "I am doing well, thank you! How can I assist you today?"},
    {"role": "user", "content": "Can you tell me a joke?"},
  ]
)

In [ ]:
completion

In [ ]:
print(completion.__verbose['prompt'])

In [ ]:
print(completion.choices[0].message.content)

#### Function calling

In [ ]:
get_weather_schema =  {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": [
                "location"
            ]
        }
    }
}

In [ ]:
def get_weather(location):
    print(f"Getting weather for {location}...")
    # Simulate a weather API call
    return f"The current temperature in {location} is 25°C."

In [ ]:
chat_history = [
        #{"role": "system", "content": "You are a helpful assistant with tool calling capabilities. When you receive a tool call response, use the output to format an answer to the orginal user question."},
        {"role": "user", "content": "Hi, what is the weather today in Bogota?"}
    ]

In [ ]:
completion = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=chat_history,
    tools=[get_weather_schema]
)

In [ ]:
print(completion.__verbose['prompt'])

In [ ]:
print(completion.choices[0].message.content)

In [ ]:
print(completion.choices[0].message.tool_calls[0].function)

In [ ]:
import ast
if completion.choices[0].message.tool_calls:
    if completion.choices[0].message.tool_calls[0].function.name == 'get_weather':
        response = get_weather(ast.literal_eval(completion.choices[0].message.tool_calls[0].function.arguments)['location'])
        print(response)

In [ ]:
response = str({"output": response}) # If I don't return it in form of this dict. The model doesn't responds correctly. It's in the Llama documentation also https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_1/
response

In [ ]:
chat_history.append({"role": "assistant", "tool_calls": completion.choices[0].message.tool_calls})
chat_history.append({"role": "tool", "content": response})

In [ ]:
chat_history

In [ ]:
chat_history = [{"role": "system", "content": "You are a helpful assistant with tool calling capabilities. When you receive a tool call response, use the output to format an answer to the orginal user question."}] + chat_history

In [ ]:
chat_history

In [ ]:
completion = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=chat_history,
    #tools=[get_weather_schema] # If I include it. The model also responds but this contaminates the prompt and could harm performance
)

In [ ]:
print(completion.__verbose['prompt'])

In [ ]:
print(completion.choices[0].message.content)

In [ ]:
print(completion.choices[0].message.tool_calls[0].function)

#### Building the Agent